In [1]:
%load_ext autoreload

In [116]:
%autoreload 2

import pandas as pd
import numpy as np

import sys
sys.path.append('../tools/')

from fangraphs_projections import get_projections
import pre_process_script as pp
from league_data import process_league_data

In [129]:

id_dict = pd.read_csv('C:/Users/mitch/work/fantasy_baseball_tools/data/SFBB Player ID Map - PLAYERIDMAP.csv')
id_dict = dict(zip( id_dict.FANTRAXID.to_list(), id_dict.IDFANGRAPHS.to_list() ))

# player_list = pd.read_csv("data/league/Fantrax-Players-Believers in BABIP (BIB).csv") # Old rosters
player_list = pd.read_csv("./data/league.csv") # New rosters
player_list.ID = player_list.ID.apply(lambda x: id_dict[x] if x in id_dict.keys() else np.nan)
player_list = player_list[player_list.ID == player_list.ID]
player_list.rename(columns={"ID": "playerid"}, inplace=True)
player_list = player_list.set_index("playerid")
player_list = player_list[["Player", "Team", "Position", "Status", "Salary",  "Age", "%D"]]
# TODO: Change ADP to use Fangraphs projected ADP

In [130]:
# names = ["steamer", "fangraphsdc", "zipsdc", "razzball", "atc", "thebat", "zips"]
names = ['atc']
av_names = []   
for n in names:
    # try:
    # Get projections from Fangraphs
    bats  = get_projections("all", "bat", n)
    pitch = get_projections("all", "pit", n)
    # Save data
    bats.to_csv(f"./data/{n}_bat.csv", index=False)
    pitch.to_csv(f"./data/{n}_pitch.csv", index=False)
    bats  = pp.pre_process_bat(bats, min_pa=0).set_index("playerid")[["POINTS", "ADP"]]
    player_list.to_csv("./data/league_data.csv")
    pitch = pp.pre_process_pitch(pitch).set_index("playerid")[["POINTS", "ADP"]]
    ranks = pitch.add(bats, fill_value=0).sort_values("POINTS")
    player_list = player_list.join(ranks, how="inner")
    player_list.rename(columns={"POINTS":"POINTS_{}".format(n)}, inplace=True)
    av_names.append("POINTS_{}".format(n))
    # except:
    #     print("{} data unavailable.".format(n))

# # Calculate totals
player_list["MEAN"] = np.mean(player_list[av_names], axis=1)
player_list["STD"] = np.std(player_list[av_names], axis=1)
player_list.sort_values("MEAN", inplace=True, ascending=False)
player_list["Rank"] = list(range(1, len(player_list)+1))
player_list["Rank_round"] = np.floor( (player_list.Rank  - 1 )/ 12 ) +  1
player_list["ADP_round"] = np.floor( (player_list.ADP  - 1 )/ 12 ) +  1
player_list["Keper_Value"] = player_list.Salary -  player_list.Rank_round
player_list["Draft_Value"] = player_list.ADP_round -  player_list.Rank_round

In [132]:
player_list

,Player,Team,Position,Status,Salary,Age,%D,POINTS_atc,ADP,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,
18401,Ronald Acuna Jr.,ATL,OF,Edgar,38,25,-,95.566393,1.000000,95.566393,0.0,1,1.0,1.0,37.0,0.0
20123,Juan Soto,SD,OF,sfernand,32,25,-,85.988105,11.220000,85.988105,0.0,2,1.0,1.0,31.0,0.0
15640,Aaron Judge,NYY,OF,BBE,31,31,-,80.069483,10.950000,80.069483,0.0,3,1.0,1.0,30.0,0.0
5361,Freddie Freeman,LAD,"1B,INF",Mash,38,34,-,79.928029,8.270000,79.928029,0.0,4,1.0,1.0,37.0,0.0
19556,Yordan Alvarez,HOU,OF,TYR,27,26,-,79.576800,16.459999,79.576800,0.0,5,1.0,2.0,26.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19206,Dakota Hudson,STL,"SP,RP",FA,2,29,-,16.881467,750.320007,16.881467,0.0,981,82.0,63.0,-80.0,-19.0
12976,Austin Hedges,PIT/TEX,C,FA,1,31,-,16.874361,999.000000,16.874361,0.0,982,82.0,84.0,-81.0,2.0
14735,Brett Phillips,LAA,OF,FA,1,29,-,16.253967,999.000000,16.253967,0.0,983,82.0,84.0,-81.0,2.0


In [135]:
# Show top 20 draft value players (only ADP inside top 300)
player_list.sort_values("Draft_Value", inplace=True, ascending=False)
player_list[player_list.ADP <= 300, ["Player", "Position", ]].head(20)

,Player,Team,Position,Status,Salary,Age,%D,POINTS_atc,ADP,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,
13145,Josh Bell,CLE/MIA,"1B,INF",FA,30,31,-,49.898768,278.679993,49.898768,0.0,141,12.0,24.0,18.0,12.0
19566,Nathaniel Lowe,TEX,"1B,INF",sfernand,29,28,-,55.566632,209.860001,55.566632,0.0,84,7.0,18.0,22.0,11.0
15507,Hunter Harvey,WSH,RP,DN,7,28,-,50.630034,276.380005,50.630034,0.0,137,12.0,23.0,-5.0,11.0
7005,Ryan Pressly,HOU,RP,Patron#5,35,34,-,49.803140,265.380005,49.803140,0.0,144,12.0,23.0,23.0,11.0
15491,J.P. Crawford,SEA,"SS,INF",JM,5,28,-,49.444295,279.410004,49.444295,0.0,149,13.0,24.0,-8.0,11.0
15362,Jeff McNeil,NYM,"2B,INF,OF",Tomy,28,31,-,47.735051,289.350006,47.735051,0.0,174,15.0,25.0,13.0,10.0
17548,Taylor Ward,LAA,OF,JM,12,29,-,52.499894,236.110001,52.499894,0.0,110,10.0,20.0,2.0,10.0
12927,Brandon Nimmo,NYM,OF,Edgar,29,30,-,56.740776,196.619995,56.740776,0.0,74,7.0,17.0,22.0,10.0
14162,Carlos Correa,MIN,"SS,INF",FA,31,29,-,50.194955,260.970001,50.194955,0.0,138,12.0,22.0,19.0,10.0


In [138]:
# show best values inside the top 10 ADP_round
player_list[player_list.ADP_round <= 10].head(20).sort_values("MEAN", ascending=False)

,Player,Team,Position,Status,Salary,Age,%D,POINTS_atc,ADP,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,
16149,Aaron Nola,PHI,SP,BBE,38,30,-,64.264886,53.590000,64.264886,0.0,26,3.0,5.0,35.0,2.0
17678,Alex Bregman,HOU,"3B,INF",BBE,36,29,-,63.703281,100.839996,63.703281,0.0,29,3.0,9.0,33.0,6.0
13774,Zach Eflin,TB,SP,sfernand,26,29,-,62.336972,83.029999,62.336972,0.0,33,3.0,7.0,23.0,4.0
17995,Logan Webb,SF,SP,Tomy,27,27,-,62.067227,57.270000,62.067227,0.0,36,3.0,5.0,24.0,2.0
10155,Mike Trout,LAA,OF,BBE,30,32,-,62.024306,71.860001,62.024306,0.0,37,4.0,6.0,26.0,2.0
22250,Logan Gilbert,SEA,SP,TCE,35,26,-,61.823561,62.860001,61.823561,0.0,40,4.0,6.0,31.0,2.0
9218,Paul Goldschmidt,STL,"1B,INF",JM,36,36,-,60.601341,82.110001,60.601341,0.0,45,4.0,7.0,32.0,3.0
16478,Kyle Schwarber,PHI,OF,Mash,32,30,-,59.749992,91.379997,59.749992,0.0,49,5.0,8.0,27.0,3.0
21390,Joe Ryan,MIN,SP,BBE,31,27,-,59.112622,84.160004,59.112622,0.0,54,5.0,7.0,26.0,2.0


In [140]:
player_list[player_list.ADP_round <= 10].sort_values("MEAN", ascending=False).head(30)

,Player,Team,Position,Status,Salary,Age,%D,POINTS_atc,ADP,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,
18401,Ronald Acuna Jr.,ATL,OF,Edgar,38,25,-,95.566393,1.000000,95.566393,0.0,1,1.0,1.0,37.0,0.0
20123,Juan Soto,SD,OF,sfernand,32,25,-,85.988105,11.220000,85.988105,0.0,2,1.0,1.0,31.0,0.0
15640,Aaron Judge,NYY,OF,BBE,31,31,-,80.069483,10.950000,80.069483,0.0,3,1.0,1.0,30.0,0.0
5361,Freddie Freeman,LAD,"1B,INF",Mash,38,34,-,79.928029,8.270000,79.928029,0.0,4,1.0,1.0,37.0,0.0
19556,Yordan Alvarez,HOU,OF,TYR,27,26,-,79.576800,16.459999,79.576800,0.0,5,1.0,2.0,26.0,1.0
19755,Shohei Ohtani,LAA,"UT,SP",BBE,38,29,-,76.708927,13.140000,76.708927,0.0,6,1.0,2.0,37.0,1.0
13611,Mookie Betts,LAD,"2B,SS,INF,OF",sfernand,39,31,-,76.559956,5.240000,76.559956,0.0,7,1.0,1.0,38.0,0.0
18345,Kyle Tucker,HOU,OF,BBE,33,26,-,73.526726,6.380000,73.526726,0.0,8,1.0,1.0,32.0,0.0
14344,Matt Olson,ATL,"1B,INF",JM,37,29,-,73.036736,17.110001,73.036736,0.0,9,1.0,2.0,36.0,1.0


In [144]:
def search_players(partial_string):
    # Convert the partial string to lowercase for case insensitivity
    partial_string = partial_string.lower()
    
    # Filter the player_list dataframe based on the partial string
    search_results = player_list[player_list['Player'].str.lower().str.contains(partial_string)]
    
    # Display the information of the search results
    if len(search_results) > 0:
        return search_results[['Player', 'Team', 'Position', 'Status', 'Salary', "MEAN", "Rank", "ADP_round"]]
    else:
        return "No matching players found."



In [157]:
search_players("Paxton")

,Player,Team,Position,Status,Salary,MEAN,Rank,ADP_round
playerid,,,,,,,,
11828,James Paxton,BOS,SP,FA,5,36.782677,462,26.0


In [33]:
player_list[(player_list.Rank_round <= 11)].sort_values("Draft_Value", ascending=False).head(13)

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
16578,Yandy Diaz,TB,"3B,INF",Edgar,24,235.53,31,92%,61.093659,58.539436,...,57.134660,54.584817,54.530640,56.864028,2.375314,56,5.0,20.0,19.0,15.0
20373,Andres Munoz,SEA,RP,TYR,13,236.22,24,94%,55.562149,55.256311,...,57.492625,55.608343,56.804627,55.963191,0.881122,67,6.0,20.0,7.0,14.0
13892,Paul Sewald,SEA,RP,DN,22,254.58,32,94%,52.024073,52.903662,...,53.401098,55.751540,51.530844,52.989732,1.372592,91,8.0,22.0,14.0,14.0
18568,Luis Arraez,MIA,"1B,2B,INF",sfernand,19,207.89,25,98%,55.582329,56.816902,...,55.397248,58.574768,60.805286,57.366215,1.860465,52,5.0,18.0,14.0,13.0
19569,David Bednar,PIT,RP,FAFO,24,209.36,28,97%,53.487404,55.208707,...,55.334512,57.126930,50.259723,54.717257,2.327939,79,7.0,18.0,17.0,11.0
21029,Jhoan Duran,MIN,RP,CR,1,218.82,25,96%,53.152773,53.369622,...,54.617743,52.469810,52.414141,53.064624,0.793858,90,8.0,19.0,-7.0,11.0
17027,Alex Verdugo,BOS,OF,Tomy,22,186.90,26,98%,52.440571,53.891278,...,52.038958,51.596721,60.483117,54.156301,3.005180,82,7.0,16.0,15.0,9.0
21132,Alexis Diaz,CIN,RP,Patron#5,20,224.42,26,97%,47.016670,49.990145,...,50.641600,53.473678,53.391789,51.123129,2.245896,115,10.0,19.0,10.0,9.0
14993,Scott Barlow,KC,RP,Pete,23,220.41,30,97%,49.452151,51.118444,...,51.369971,52.240998,53.047144,51.467757,1.102930,109,10.0,19.0,13.0,9.0


# Positional Rankings

In [154]:
player_list[(player_list.Position.str.contains("1B"))].sort_values("Rank").head(20)

,Player,Team,Position,Status,Salary,Age,%D,POINTS_atc,ADP,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,
5361,Freddie Freeman,LAD,"1B,INF",Mash,38,34,-,79.928029,8.270000,79.928029,0.0,4,1.0,1.0,37.0,0.0
14344,Matt Olson,ATL,"1B,INF",JM,37,29,-,73.036736,17.110001,73.036736,0.0,9,1.0,2.0,36.0,1.0
11579,Bryce Harper,PHI,"1B,INF",TCE,32,31,-,72.793877,16.860001,72.793877,0.0,11,1.0,2.0,31.0,1.0
19611,Vladimir Guerrero Jr.,TOR,"1B,INF",Mash,31,24,-,69.367151,33.320000,69.367151,0.0,15,2.0,3.0,29.0,1.0
16578,Yandy Diaz,TB,"1B,3B,INF",Edgar,30,32,-,63.655374,139.889999,63.655374,0.0,30,3.0,12.0,27.0,9.0
19251,Pete Alonso,NYM,"1B,INF",Pete,29,28,-,63.386995,27.680000,63.386995,0.0,31,3.0,3.0,26.0,0.0
18568,Luis Arraez,MIA,"1B,2B,INF",BBE,24,26,-,61.492128,163.860001,61.492128,0.0,42,4.0,14.0,20.0,10.0
9218,Paul Goldschmidt,STL,"1B,INF",JM,36,36,-,60.601341,82.110001,60.601341,0.0,45,4.0,7.0,32.0,3.0
20529,Nolan Jones,COL,"1B,INF,OF",sfernand,3,25,-,59.773971,56.950001,59.773971,0.0,48,4.0,5.0,-1.0,1.0


In [155]:
player_list[(player_list.Position.str.contains("2B"))].sort_values("Rank").head(20)

,Player,Team,Position,Status,Salary,Age,%D,POINTS_atc,ADP,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,
13611,Mookie Betts,LAD,"2B,SS,INF,OF",sfernand,39,31,-,76.559956,5.240000,76.559956,0.0,7,1.0,1.0,38.0,0.0
5417,Jose Altuve,HOU,"2B,INF",Mash,31,33,-,62.275418,43.380001,62.275418,0.0,34,3.0,4.0,28.0,1.0
12533,Marcus Semien,TEX,"2B,INF",Patron#5,36,33,-,61.693464,30.490000,61.693464,0.0,41,4.0,3.0,32.0,-1.0
18568,Luis Arraez,MIA,"1B,2B,INF",BBE,24,26,-,61.492128,163.860001,61.492128,0.0,42,4.0,14.0,20.0,10.0
16556,Ozzie Albies,ATL,"2B,INF",JM,31,26,-,59.934054,23.240000,59.934054,0.0,47,4.0,2.0,27.0,-2.0
13613,Ketel Marte,ARI,"2B,INF",Tomy,30,30,-,58.325370,121.430000,58.325370,0.0,61,6.0,11.0,24.0,5.0
16997,Gleyber Torres,NYY,"2B,INF",SS268,25,26,-,56.945605,89.190002,56.945605,0.0,71,6.0,8.0,19.0,2.0
21479,Nico Hoerner,CHC,"2B,SS,INF",sfernand,23,26,-,56.377684,65.239998,56.377684,0.0,78,7.0,6.0,16.0,-1.0
29695,Matt McLain,CIN,"2B,SS,INF",DN,3,24,-,53.940311,62.650002,53.940311,0.0,97,9.0,6.0,-6.0,-3.0


In [156]:
player_list[(player_list.Position.str.contains("3B"))].sort_values("Rank").head(20)


,Player,Team,Position,Status,Salary,Age,%D,POINTS_atc,ADP,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,
13510,Jose Ramirez,CLE,"3B,INF",Mash,37,31,-,70.479289,15.000000,70.479289,0.0,12,1.0,2.0,36.0,1.0
17350,Rafael Devers,BOS,"3B,INF",Tomy,34,27,-,68.393786,24.299999,68.393786,0.0,17,2.0,2.0,32.0,0.0
18360,Austin Riley,ATL,"3B,INF",TCE,29,26,-,67.805272,20.139999,67.805272,0.0,19,2.0,2.0,27.0,0.0
25764,Bobby Witt Jr.,KC,"3B,SS,INF",DN,18,23,-,66.758512,2.460000,66.758512,0.0,22,2.0,1.0,16.0,-1.0
17678,Alex Bregman,HOU,"3B,INF",BBE,36,29,-,63.703281,100.839996,63.703281,0.0,29,3.0,9.0,33.0,6.0
16578,Yandy Diaz,TB,"1B,3B,INF",Edgar,30,32,-,63.655374,139.889999,63.655374,0.0,30,3.0,12.0,27.0,9.0
11493,Manny Machado,SD,"3B,INF",Patron#5,35,31,-,58.072729,70.000000,58.072729,0.0,62,6.0,6.0,29.0,0.0
26289,Gunnar Henderson,BAL,"3B,SS,INF",BBE,6,22,-,56.875425,31.299999,56.875425,0.0,72,6.0,3.0,0.0,-3.0
20437,Royce Lewis,MIN,"3B,SS,INF",Pete,6,24,-,55.210666,53.700001,55.210666,0.0,89,8.0,5.0,-2.0,-3.0


In [158]:
player_list[(player_list.Position.str.contains("SS"))].sort_values("Rank").head(20)

,Player,Team,Position,Status,Salary,Age,%D,POINTS_atc,ADP,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,
13611,Mookie Betts,LAD,"2B,SS,INF,OF",sfernand,39,31,-,76.559956,5.240000,76.559956,0.0,7,1.0,1.0,38.0,0.0
13624,Corey Seager,TEX,"SS,INF",Tomy,34,29,-,68.843560,22.760000,68.843560,0.0,16,2.0,2.0,32.0,0.0
19709,Fernando Tatis Jr.,SD,"SS,INF,OF",TCE,29,24,-,67.786357,8.510000,67.786357,0.0,20,2.0,1.0,27.0,-1.0
25764,Bobby Witt Jr.,KC,"3B,SS,INF",DN,18,23,-,66.758512,2.460000,66.758512,0.0,22,2.0,1.0,16.0,-1.0
16252,Trea Turner,PHI,"SS,INF",TCE,37,30,-,62.233543,12.780000,62.233543,0.0,35,3.0,1.0,34.0,-2.0
12916,Francisco Lindor,NYM,"SS,INF",Tomy,37,30,-,58.441124,24.299999,58.441124,0.0,59,5.0,2.0,32.0,-3.0
19612,Bo Bichette,TOR,"SS,INF",SS268,29,25,-,58.394964,38.919998,58.394964,0.0,60,5.0,4.0,24.0,-1.0
12161,Xander Bogaerts,SD,"SS,INF",sfernand,34,31,-,57.138764,110.239998,57.138764,0.0,69,6.0,10.0,28.0,4.0
26289,Gunnar Henderson,BAL,"3B,SS,INF",BBE,6,22,-,56.875425,31.299999,56.875425,0.0,72,6.0,3.0,0.0,-3.0


In [159]:
player_list[(player_list.Position.str.contains("OF"))].sort_values("Rank").head(60)

,Player,Team,Position,Status,Salary,Age,%D,POINTS_atc,ADP,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,
18401,Ronald Acuna Jr.,ATL,OF,Edgar,38,25,-,95.566393,1.000000,95.566393,0.0,1,1.0,1.0,37.0,0.0
20123,Juan Soto,SD,OF,sfernand,32,25,-,85.988105,11.220000,85.988105,0.0,2,1.0,1.0,31.0,0.0
15640,Aaron Judge,NYY,OF,BBE,31,31,-,80.069483,10.950000,80.069483,0.0,3,1.0,1.0,30.0,0.0
19556,Yordan Alvarez,HOU,OF,TYR,27,26,-,79.576800,16.459999,79.576800,0.0,5,1.0,2.0,26.0,1.0
13611,Mookie Betts,LAD,"2B,SS,INF,OF",sfernand,39,31,-,76.559956,5.240000,76.559956,0.0,7,1.0,1.0,38.0,0.0
18345,Kyle Tucker,HOU,OF,BBE,33,26,-,73.526726,6.380000,73.526726,0.0,8,1.0,1.0,32.0,0.0
25878,Corbin Carroll,ARI,OF,JM,8,23,-,69.391598,4.490000,69.391598,0.0,14,2.0,1.0,6.0,-1.0
23697,Julio Rodriguez,SEA,OF,BBE,8,22,-,67.910945,2.840000,67.910945,0.0,18,2.0,1.0,6.0,-1.0
19709,Fernando Tatis Jr.,SD,"SS,INF,OF",TCE,29,24,-,67.786357,8.510000,67.786357,0.0,20,2.0,1.0,27.0,-1.0


In [39]:
# SP
player_list[(player_list.Position.str.contains("SP")) & (~(player_list.Position.str.contains("RP")))].sort_values("Rank").head(60)


,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
19755,Shohei Ohtani,LAA,"UT,SP",uu,27,8.38,28,100%,130.445790,128.597540,...,133.815622,134.239918,135.161226,131.183120,3.641907,1,1.0,1.0,26.0,0.0
13125,Gerrit Cole,NYY,SP,SS268,30,12.40,32,100%,71.948656,71.855404,...,75.110939,69.547964,73.232936,72.082469,1.762380,10,1.0,1.0,29.0,0.0
10954,Jacob deGrom,TEX,SP,DN,30,29.68,34,100%,76.076777,75.398107,...,70.632751,71.983880,61.124707,71.657592,5.083546,12,1.0,3.0,29.0,2.0
19361,Corbin Burnes,MIL,SP,uu,23,11.71,28,100%,68.445391,68.810137,...,73.639801,67.097206,71.550904,69.657918,2.226998,16,2.0,1.0,21.0,-1.0
16149,Aaron Nola,PHI,SP,TYR,29,24.28,29,100%,66.334693,66.749607,...,71.081265,71.319313,71.634256,69.096007,2.278309,17,2.0,2.0,27.0,0.0
3137,Max Scherzer,NYM,SP,CR,30,37.57,38,100%,70.783471,69.368009,...,70.418333,67.791796,64.643157,68.567363,2.041542,18,2.0,4.0,28.0,2.0
8700,Justin Verlander,NYM,SP,Pete,30,36.02,40,100%,67.212963,66.725169,...,68.899955,67.913617,70.534499,68.205033,1.240738,19,2.0,3.0,28.0,1.0
19427,Shane Bieber,CLE,SP,DN,27,41.04,27,100%,62.016861,63.828243,...,64.831765,58.933659,67.059188,63.723882,2.646508,29,3.0,4.0,24.0,1.0
21483,Shane McClanahan,TB,SP,sfernand,18,34.52,25,100%,65.042189,63.952661,...,64.225271,61.197905,64.073932,63.399030,1.368091,30,3.0,3.0,15.0,0.0


In [40]:
# RP
player_list[(player_list.Position.str.contains("RP")) & (~(player_list.Position.str.contains("SP")))].sort_values("Rank").head(20)


,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
21032,Emmanuel Clase,CLE,RP,TYR,17,62.14,25,100%,59.618512,60.834925,...,63.274836,62.074124,64.085468,61.914772,1.478995,34,3.0,6.0,14.0,3.0
14212,Josh Hader,SD,RP,uu,27,77.61,28,100%,59.541839,61.743191,...,59.294238,58.963994,62.355237,60.910143,1.736252,38,4.0,7.0,23.0,3.0
7005,Ryan Pressly,HOU,RP,Patron#5,28,112.79,34,100%,54.464551,56.526186,...,56.192865,56.735991,56.488569,56.494729,1.192910,62,6.0,10.0,22.0,4.0
20666,Felix Bautista,BAL,RP,uu,28,148.97,27,100%,53.972107,56.041738,...,56.992081,55.787262,58.683165,56.398520,1.427517,63,6.0,13.0,22.0,7.0
20373,Andres Munoz,SEA,RP,TYR,13,236.22,24,94%,55.562149,55.256311,...,57.492625,55.608343,56.804627,55.963191,0.881122,67,6.0,20.0,7.0,14.0
18138,Ryan Helsley,STL,RP,uu,11,131.94,28,100%,56.407731,56.235764,...,55.617388,58.081994,49.595905,55.303160,2.671336,73,7.0,11.0,4.0,4.0
15816,Devin Williams,MIL,RP,DN,26,93.28,28,100%,52.403362,54.206398,...,55.932710,56.251806,56.486934,55.168390,1.436534,74,7.0,8.0,19.0,1.0
16122,Jordan Romano,TOR,RP,Pete,26,100.43,29,100%,54.144394,54.055656,...,56.615459,56.257766,54.086501,54.868354,1.114142,77,7.0,9.0,19.0,2.0
19569,David Bednar,PIT,RP,FAFO,24,209.36,28,97%,53.487404,55.208707,...,55.334512,57.126930,50.259723,54.717257,2.327939,79,7.0,18.0,17.0,11.0


In [41]:
player_list[player_list.Player.str.contains("CJ")].sort_values("Rank").head(20)

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
25768,CJ Abrams,WSH,"SS,INF",uu,8,221.89,22,95%,34.863134,38.269022,...,33.91787,31.906921,41.566513,36.394729,3.188113,437,37.0,19.0,-29.0,-18.0


In [42]:

player_list.sort_values("MEAN", inplace=True, ascending=False)

player_list = player_list[player_list.Rank_round <= 30]
player_list_keper = player_list.dropna()

In [43]:
player_list

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
19755,Shohei Ohtani,LAA,"UT,SP",uu,27,8.38,28,100%,130.445790,128.597540,...,133.815622,134.239918,135.161226,131.183120,3.641907,1,1.0,1.0,26.0,0.0
20123,Juan Soto,SD,OF,sfernand,25,6.64,24,100%,88.385991,84.217356,...,85.038777,83.252415,83.528776,83.872503,2.827398,2,1.0,1.0,24.0,0.0
15640,Aaron Judge,NYY,OF,uu,28,2.21,30,100%,81.578427,82.748481,...,81.720702,82.837946,84.762255,82.464632,1.196697,3,1.0,1.0,27.0,0.0
5361,Freddie Freeman,LAD,"1B,INF",FAFO,30,13.91,33,100%,77.085391,78.808044,...,81.615794,78.052348,80.243817,78.761034,1.715683,4,1.0,2.0,29.0,1.0
19611,Vladimir Guerrero Jr.,TOR,"1B,INF",FAFO,25,12.91,24,100%,79.878568,74.760641,...,76.646933,78.667670,75.534852,75.737981,3.506502,5,1.0,1.0,24.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27465,Spencer Torkelson,DET,"1B,INF",uu,17,269.22,23,87%,35.365132,39.055795,...,36.202074,34.802114,46.776741,38.703853,4.075547,356,30.0,23.0,-13.0,-7.0
13621,Jeimer Candelario,WSH,"3B,INF",FA,1,342.88,29,49%,36.205239,39.031366,...,35.948550,34.745320,46.131448,38.685365,3.798946,357,30.0,29.0,-29.0,-1.0
16933,Elieser Hernandez,NYM,"SP,RP",FA,1,867.40,27,4%,40.518896,39.313730,...,34.118870,35.900045,44.292876,38.680010,3.275506,358,30.0,73.0,-29.0,43.0


In [44]:
player_list_keper.Status.value_counts()

FA          62
Tomy        28
sfernand    26
Pete        26
uu          25
FAFO        25
SS268       25
TCE         25
DN          24
Patron#5    24
TYR         23
CR          23
Edgar       22
W (Tue)      1
W (Thu)      1
Name: Status, dtype: int64

In [45]:
player_list_keper[player_list_keper.Status == "Canha Ge"].sort_values("Keper_Value", ascending=False).head(50)

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,


In [46]:
player_list.to_csv('data/proc/league_player_list_with_ranks.csv', index=True, index_label='IDFANGRAPHS')

In [47]:
rookies = ["23697", "26288", "sa3008130", "22810"]
team = player_list[player_list.Status == "TDRJ"]
team = team.loc[~team.index.isin(rookies)]
team["adj_sal"] = 100 * (31 - team.Salary) / 31 

In [48]:
import cvxpy

N = 11


utilities = np.array(team.MEAN - team.adj_sal) 

# The variable we are solving for
selection = cvxpy.Variable(len(utilities),  boolean = True)


# The sum of the weights should be less than or equal to P

number_constraint = sum( selection ) == N


# Our total utility is the sum of the item utilities
total_utility = utilities @ selection

# We tell cvxpy that we want to maximize total utility 
# subject to weight_constraint. All constraints in 
# cvxpy must be passed as a list
knapsack_problem = cvxpy.Problem(cvxpy.Maximize(total_utility), [number_constraint])

# Solving the problem
knapsack_problem.solve(solver=cvxpy.GLPK_MI)

team.loc[[ i == 1.0 for i in selection.value]]#, ["Player", "Position", "Salary", cost]]

ValueError: Invalid dimensions (0,).

In [ ]:
team.MEAN - team.adj_sal

playerid
19755    45.658202
15640    -4.419444
18345     0.561265
10155   -26.931699
19361     7.489677
14710   -25.979547
14212   -20.280090
12916   -37.034773
16578    20.731312
18138    51.769104
22250    48.130467
15890     8.198726
12861   -40.244058
20454    -1.916672
14161     0.841512
15474   -12.531355
20538    45.222970
17919    -9.678165
14524    43.295693
14527    41.378103
19447     0.712605
17479   -16.475028
19316    36.285319
10061    10.368600
dtype: float64

In [ ]:
team.MEAN / np.max(team.MEAN)

playerid
19755    1.000000
15640    0.638291
18345    0.552223
10155    0.539203
19361    0.531000
14710    0.496746
14212    0.466035
12916    0.436300
16578    0.433996
18138    0.424575
22250    0.396484
15890    0.387050
12861    0.386620
20454    0.383668
14161    0.380058
15474    0.376432
20538    0.374038
17919    0.373556
14524    0.359158
14527    0.344354
19447    0.329255
17479    0.321082
19316    0.305037
10061    0.304185
Name: MEAN, dtype: float64